In [ ]:
%matplotlib inline
import numpy as np
from parametric import Attribute, parametrize
from optimistic import experiment
from MOTorNOT import GratingMOT, QuadrupoleCoils
from MOTorNOT.integration import Solver, generate_initial_conditions

linewidth = 2*np.pi*29e6

@parametrize
class MOT:
    coil_current = Attribute('coil_current', 30)
    coil_delta = Attribute('coil_delta', -2)
    
    detuning = Attribute('detuning', -1.8*linewidth)
#     detuning = Attribute('detuning', -0.5*linewidth)

    alpha = Attribute('alpha', 30)
#     radius = Attribute('radius', 10e-3)
#     power = Attribute('power', 20e-3)
    radius = Attribute('radius', 40e-3)
    power = Attribute('power', 10e-3 * 16)
    sectors = Attribute('sectors', 3)
    cutoff = Attribute('cutoff', 10e-3)
    position = Attribute('position', 9e-3)
    beam_position = Attribute('beam_position', 0)
    
    def make_gmot(self):
        field = QuadrupoleCoils(radius=0.1, 
                                offset=0.1, 
                                turns=100, 
                                current=self.coil_current(), 
                                axis=2, deltaI=self.coil_delta()).field
        gmot = GratingMOT(position = self.position(), 
                          alpha = self.alpha(), 
                          detuning = self.detuning(), 
                          radius = self.radius(), 
                          power = self.power(), 
                          handedness = -1, 
                          R1 = 1/self.sectors(), 
                          field = field,
                          sectors=self.sectors(),
                          beam_type='gaussian',
                          grating_radius=self.cutoff())
        return gmot 
    
    @experiment(parallel=True)
    def capture_velocity(self):
        gmot = self.make_gmot()
        X0, V0 = generate_initial_conditions(-self.position(), np.linspace(1, 100, 400), phi=90, theta=0)
        sol = Solver(gmot.acceleration, X0, V0).run(20e-2, dt=1e-5)           
        vi, vf = sol.capture_velocity()
        return vf

mot = MOT()

In [ ]:
mot.make_gmot().plot('xz', numpoints=50, limits=([-.01, .01], [-.008, .009]), quiver_scale=50, component='radial', midpoint=.5)

# mot.make_gmot().plot('xz', numpoints=80, limits=([-.003, .003], [-.002, .005]), quiver_scale=50, component='radial')   # uniform trap center
# mot.make_gmot().plot('xz', numpoints=50, limits=([-.002, .002], [-.005, .004]), quiver_scale=50, component='radial', midpoint=1)  # gaussian trap center


In [ ]:
def axial_projection(x, v):
    ''' Returns the axial projection of a vector v at position x '''
    X = np.atleast_2d(x)
    V = np.atleast_2d(v)

    zhat = np.array([0, 0, np.sign(X[:, 2])])

    return (zhat*V).sum(axis=1)    

axial_projection([0, 0, -1], [0, 0, -1])

In [ ]:
## run a 1D study
from optimistic.algorithms import ParallelGridSearch
# grid = ParallelGridSearch.study(mot.capture_velocity, mot.detuning, (-5*linewidth, 0), steps=50)
grid = ParallelGridSearch.study(mot.capture_velocity, mot.coil_delta, (-10, 5), steps=50)

In [ ]:
grid.data

In [ ]:
from optimistic.algorithms import ParallelGridSearch

grid = ParallelGridSearch(mot.capture_velocity, steps=50)
grid.add_parameter(mot.alpha, (15, 60))
grid.add_parameter(mot.sectors, points=[3, 4, 6, 8])
grid.run()

In [ ]:
import matplotlib.pyplot as plt

def plot_curves(self, x, y):
    z = self.experiment.__name__
    for x0, df in self.data.groupby(y):
        plt.plot(df[x], df[z], label=f'{y}={np.round(x0, 3)}')
    plt.legend(loc=(1.04,0))
    plt.xlabel(x)
    plt.ylabel(z)
plot_curves(grid, 'detuning', 'coil_current')

In [ ]:
grid.plot.curves('alpha', 'sectors')

In [ ]:
-3.346758e+08/linewidth